In [ ]:
!pip install tokenizers

In [ ]:
## importing the tokenizer and subword BPE trainer
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordLevel, WordPiece
from tokenizers.trainers import BpeTrainer, WordLevelTrainer, \
                                WordPieceTrainer, UnigramTrainer
from tokenizers.pre_tokenizers import Whitespace

In [ ]:
!wget http://www.gutenberg.org/cache/epub/16457/pg16457.txt

--2023-12-06 17:06:51--  http://www.gutenberg.org/cache/epub/16457/pg16457.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/cache/epub/16457/pg16457.txt [following]
--2023-12-06 17:06:51--  https://www.gutenberg.org/cache/epub/16457/pg16457.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 617780 (603K) [text/plain]
Saving to: ‘pg16457.txt.1’

pg16457.txt.1       100%[===================>] 603.30K  --.-KB/s    in 0.08s   

2023-12-06 17:07:12 (6.94 MB/s) - ‘pg16457.txt.1’ saved [617780/617780]



In [ ]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
!unzip wikitext-103-raw-v1.zip

--2023-12-06 17:07:12--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.54.64, 52.216.44.120, 52.217.90.214, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.54.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191984949 (183M) [application/zip]
Saving to: ‘wikitext-103-raw-v1.zip.1’

wikitext-103-raw-v1 100%[===================>] 183.09M  61.8MB/s    in 3.0s    

2023-12-06 17:07:16 (61.8 MB/s) - ‘wikitext-103-raw-v1.zip.1’ saved [191984949/191984949]

Archive:  wikitext-103-raw-v1.zip
replace wikitext-103-raw/wiki.test.raw? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
unk_token = "<UNK>"  # token for unknown words
spl_tokens = ["<UNK>", "<SEP>", "<MASK>", "<CLS>"]  # special tokens

def prepare_tokenizer_trainer(alg):
    """
    Prepares the tokenizer and trainer with unknown & special tokens.
    """
    if alg == 'BPE':
        tokenizer = Tokenizer(BPE(unk_token = unk_token))
        trainer = BpeTrainer(special_tokens = spl_tokens)
    elif alg == 'UNI':
        tokenizer = Tokenizer(Unigram())
        trainer = UnigramTrainer(unk_token= unk_token, special_tokens = spl_tokens)
    elif alg == 'WPC':
        tokenizer = Tokenizer(WordPiece(unk_token = unk_token))
        trainer = WordPieceTrainer(special_tokens = spl_tokens)
    else:
        tokenizer = Tokenizer(WordLevel(unk_token = unk_token))
        trainer = WordLevelTrainer(special_tokens = spl_tokens)

    tokenizer.pre_tokenizer = Whitespace()
    return tokenizer, trainer


def train_tokenizer(files, alg='WLV'):
    """
    Takes the files and trains the tokenizer.
    """
    tokenizer, trainer = prepare_tokenizer_trainer(alg)
    tokenizer.train(files, trainer) # training the tokenzier
    tokenizer.save("./tokenizer-trained.json")
    tokenizer = Tokenizer.from_file("./tokenizer-trained.json")
    return tokenizer

def tokenize(input_string, tokenizer):
    """
    Tokenizes the input string using the tokenizer provided.
    """
    output = tokenizer.encode(input_string)
    return output


In [ ]:
small_file = ['pg16457.txt']
large_files = [f"./wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]

tokens_dict = {}

for files in [small_file, large_files]:
    print(f"========Using vocabulary from {files}=======")
    for alg in ['WLV', 'BPE', 'UNI', 'WPC']:
        trained_tokenizer = train_tokenizer(files, alg)
        input_string = "This is a deep learning tokenization tutorial. Tokenization is the first step in a deep learning NLP pipeline. We will be comparing the tokens generated by each tokenization model. Excited much?!😍"
        output = tokenize(input_string, trained_tokenizer)
        tokens_dict[alg] = output.tokens
        print("----", alg, "----")
        print(output.tokens, "->", len(output.tokens))

========Using vocabulary from ['pg16457.txt']=======
---- WLV ----
['This', 'is', 'a', 'deep', 'learning', '<UNK>', '<UNK>', '.', '<UNK>', 'is', 'the', 'first', 'step', 'in', 'a', 'deep', 'learning', '<UNK>', '<UNK>', '.', 'We', 'will', 'be', 'comparing', 'the', '<UNK>', 'generated', 'by', 'each', '<UNK>', 'model', '.', '<UNK>', 'much', '<UNK>'] -> 35
---- BPE ----
['This', 'is', 'a', 'deep', 'learning', 'to', 'ken', 'ization', 't', 'ut', 'or', 'ial', '.', 'T', 'ok', 'en', 'ization', 'is', 'the', 'first', 'step', 'in', 'a', 'deep', 'learning', 'N', 'L', 'P', 'pi', 'pe', 'line', '.', 'We', 'will', 'be', 'comparing', 'the', 'to', 'k', 'ens', 'generated', 'by', 'each', 'to', 'ken', 'ization', 'model', '.', 'Ex', 'c', 'ited', 'much', '?', '!', '<UNK>'] -> 55
---- UNI ----
['Thi', 's', 'is', 'a', 'deep', 'learn', 'ing', 'to', 'ken', 'iz', 'ation', 't', 'u', 'to', 'rial', '.', 'To', 'ken', 'iz', 'ation', 'is', 'the', 'fir', 's', 't', 'step', 'in', 'a', 'deep', 'learn', 'ing', 'N', 'L', 'P', 

In [ ]:

tokens_dict = {}

for alg in ['BPE', 'UNI', 'WPC']:
    trained_tokenizer = train_tokenizer(large_files, alg)
    input_string = "This is a deep learning tokenization tutorial. Tokenization is the first step in a deep learning NLP pipeline. We will be comparing the tokens generated by each tokenization model. Excited much?!😍"
    output = tokenize(input_string, trained_tokenizer)
    tokens_dict[alg] = output.tokens